### Forecasting Mini-Course Sales
特徴量生成 + 基本的に2020年は削除  
https://www.kaggle.com/code/gauthamupadhyaya/mini-course-sales
- <b>学習は2021年を予測</b> (テスト予測には2021年も学習に使う)
- 休日情報も使う
- 小さい値の追従性も鑑み、対数で学習させる
- コロナ明け需要から、係数を掛ける　1.2倍(?)
- ~~国、店舗、製品を分けて学習してみる~~
- ~~前年の売上げを特徴量として組み込んでみる~~

In [1]:
import numpy as np
import pandas as pd
from holidays import CountryHoliday
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
import make_graph    # 自前
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
df_train = pd.read_csv("./data/train.csv", parse_dates=["date"])
df_test = pd.read_csv("./data/test.csv", parse_dates=["date"])
df_subm = pd.read_csv("./data/sample_submission.csv")

In [4]:
# 一旦、学習用テスト用をまとめる
df_test["num_sold"] = np.nan
df_all = pd.concat([df_train, df_test], ignore_index=True)

df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day'] = df_all['date'].dt.day
print(df_all.shape)
df_all.head(5)

(164325, 9)


,id,date,country,store,product,num_sold,year,month,day
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59.0,2017,1,1
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49.0,2017,1,1


In [121]:
# 資料用
df_buf = df_train.copy()
df_buf['product'] = df_buf['product'].str.replace('Using LLMs to ', '')

In [3]:
# make_graph.trends(df_train, "num_sold", 1200, 300, "graph0.html", "origin")

In [5]:
CSP = {col: list(df_all[col].unique()) for col in ["country", "store", "product"]}
CSP

{'country': ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'],
 'store': ['Kaggle Learn', 'Kaggle Store', 'Kagglazon'],
 'product': ['Using LLMs to Improve Your Coding',
  'Using LLMs to Train More LLMs',
  'Using LLMs to Win Friends and Influence People',
  'Using LLMs to Win More Kaggle Competitions',
  'Using LLMs to Write Better']}

In [6]:
def select_csp(df, idx):
    """インデックス指定で各名称取得、DataFrameフィルタ"""
    if len(idx) != 3:
        return (), pd.DataFrame()
    df_ret = df.copy()
    for i, col in zip(idx, ["country", "store", "product"]):
        if i < 0:
            continue
        idx_buf = i
        if i >= len(CSP[col]):
            print(f"[error] 指定インデックスが範囲外です。 ({col}: {CSP[col]})")
            idx_buf = 0
        df_ret = df_ret[df_ret[col] == CSP[col][idx_buf]]

    sel = {col: list(df_ret[col].unique()) for col in ["country", "store", "product"]}
    return sel, df_ret

In [7]:
# 出力テスト
csp, df_buf = select_csp(df_all, [1, 0, -1])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]}"
p = make_graph.make_trend(df_buf, "num_sold", "product", 900, 300, ttl)
show(p)

In [8]:
def make_dummies(df_temp, cat_cols):
    """国、店舗、製品情報をダミー変数化"""
    df_dummy = pd.get_dummies(df_temp[cat_cols]).astype(np.uint8)
    df_temp = pd.merge(df_temp, df_dummy, left_index=True, right_index=True)
    print("dummies:", df_temp.shape)
    return df_temp

In [9]:
def datetime_features(df_temp):
    """時間情報の特徴量生成"""
    df_temp['dayofyear'] = df_temp['date'].dt.day_of_year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    # df_temp['weekofyear'] = df_temp['date'].dt.weekofyear  # 見つからない
    df_temp['weekofyear'] = df_temp['date'].apply(lambda x: x.isocalendar()[1])
    df_temp['is_weekend'] = (df_temp['dayofweek'] >= 5).astype(np.int8)
    df_temp['is_month_start'] = df_temp['date'].dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp['date'].dt.is_month_end.astype(np.int8)
    df_temp['dayname'] = df_temp['date'].dt.strftime("%A")    # 曜日。後でダミー変数化
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)    
    print("datetime:", df_temp.shape)
    return df_temp

In [10]:
def GetAndAddHolidays(df):
    # Get country-specific holidays
    country_list = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']
    years = np.arange(df['year'].min(), df['year'].max() + 1)
    holidays = pd.DataFrame(columns=['date', 'holiday', 'country'])

    for country in country_list:
        for h in CountryHoliday(country, years=years).items():
            i=len(holidays)
            holidays.loc[i,'date']=pd.to_datetime(h[0])
            holidays.loc[i,'holiday']=h[1]
            holidays.loc[i,'country']=country

    # Add common holidays from Dec 24 to Dec 31 for each country
    common_holidays = pd.DataFrame({
    'date': pd.to_datetime(['2017-04-16','2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31',
                            '2018-04-01','2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31',
                            '2019-04-21','2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31',
                            '2020-04-12','2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
                            '2021-04-04','2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
                            '2022-04-17','2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31']),
    'holiday': 'Holiday Season'
    })

    country_common_holidays = pd.DataFrame()
    for country in country_list:
        temp_common_holidays = common_holidays.copy()
        temp_common_holidays['country'] = country
        country_common_holidays = pd.concat([country_common_holidays, temp_common_holidays], ignore_index=True)

    holidays = pd.concat([holidays, country_common_holidays], ignore_index=True)
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays['isHoliday']=1
    
    
    # Merge holidays with the original DataFrame based on date and country
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df['holiday'] = df['holiday'].fillna('Not Holiday')
    df['isHoliday'] = df['isHoliday'].fillna(0)
    
    # Add Weekend columns
    #df['Friday'] = (df['dayofweek'] == 4).astype(int)
    #df['Saturday'] = (df['dayofweek'] == 5).astype(int)
    #df['Sunday'] = (df['dayofweek'] == 6).astype(int)
    
    #Quarter, weekly season
    #df['Q1'] = ((df['month'] >= 1) & (df['month'] <= 3)).astype(int)
    #df['Q2'] = ((df['month'] >= 4) & (df['month'] <= 6)).astype(int)
    #df['Q3'] = ((df['month'] >= 7) & (df['month'] <= 9)).astype(int)
    #df['Q4'] = ((df['month'] >= 10) & (df['month'] <= 12)).astype(int)

    # Remove duplicates from df
    df = df.drop_duplicates(subset=['date', 'country','product','store'], keep='first').reset_index().drop(columns = ["index"])

    print("holidays:", df.shape)
    return df, holidays

#full, Holidays = GetAndAddHolidays(full)

In [11]:
def seasonality_features(df_temp):
    df_temp['month_sin'] = np.sin(2*np.pi*df_temp["month"]/12)
    df_temp['month_cos'] = np.cos(2*np.pi*df_temp["month"]/12)
    df_temp['day_sin'] = np.sin(2*np.pi*df_temp["day"]/24)    # 24 -> 30 -> 24
    df_temp['day_cos'] = np.cos(2*np.pi*df_temp["day"]/24)
    print("seasonality:", df_temp.shape)
    return df_temp

In [12]:
def make_features(df_temp):
    """全特徴量を追加"""
    df_temp = datetime_features(df_temp)
    df_temp, TrainHolidays = GetAndAddHolidays(df_all)
    df_temp = make_dummies(df_temp, ["country", "store", "product", "dayname", "holiday"])
    df_temp = seasonality_features(df_temp)
    return df_temp

# 特徴量生成
df_feat = make_features(df_all)
df_feat.head(3)

datetime: (164325, 21)
holidays: (164325, 23)
dummies: (164325, 129)
seasonality: (164325, 133)


,id,date,country,store,product,num_sold,year,month,day,dayofyear,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926


各モデルの性能確認

In [13]:
import time
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ElasticNet
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [14]:
def get_smape(actual, forecast):
    """自前で評価値(SMAPE:https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)産出"""
    smape = 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(forecast) + np.abs(actual)))
    # smape = np.sum(np.abs(forecast - actual)) / np.sum(forecast + actual)
    return smape

学習用、検証用、テスト用にデータ分離 (2020年を削除)

In [15]:
# 学習と検証用に分離 (2020年削除)
remove_cols = ["id", "date", "country", "store", "product", "num_sold", "dayname", "holiday"]
train = df_feat[df_feat["year"]<2020].copy()
valid = df_feat[df_feat["year"]==2021].copy()

X_train = train.drop(columns=remove_cols)
y_train = np.log(train["num_sold"].values)    # 小さい値のために対数で学習させる (後で戻す)
X_valid = valid.drop(columns=remove_cols)
y_valid = np.log(valid["num_sold"].values)
print(X_train.shape, len(y_train), X_valid.shape, len(y_valid))

(82125, 125) 82125 (27375, 125) 27375


In [16]:
# テスト用 (2020年削除)
Xt_train = df_feat[~(df_feat["year"].isin([2020,2022]))].drop(columns=remove_cols)
yt_train = np.log(df_feat[~(df_feat["year"].isin([2020,2022]))]["num_sold"].values)
X_test = df_feat[df_feat["year"]==2022].drop(columns=remove_cols) 
print(Xt_train.shape, len(yt_train), X_test.shape)

(109500, 125) 109500 (27375, 125)


In [17]:
X_train.tail(3)

,year,month,day,dayofyear,dayofweek,quarter,weekofyear,is_weekend,is_month_start,is_month_end,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
82122,2019,12,31,365,1,4,1,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819
82123,2019,12,31,365,1,4,1,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819
82124,2019,12,31,365,1,4,1,0,0,1,...,0,0,0,0,0,0,-2.449294e-16,1.0,0.965926,-0.258819


In [18]:
yt_train

array([4.14313473, 4.18965474, 2.19722458, ..., 4.7095302 , 6.46302946,
       6.28971557])

In [19]:
regressors = [LinearRegression(), 
              Lasso(), 
              DecisionTreeRegressor(), 
              Ridge(), 
              ElasticNet(), 
              RandomForestRegressor(), 
              XGBRegressor(), 
              CatBoostRegressor(), 
              LGBMRegressor(), 
              GradientBoostingRegressor(), 
              HistGradientBoostingRegressor()]

In [86]:
scores = []
for model in regressors:
    name = str(model)[:-2]
    name = "XGBRegressor" if "XGBRegressor" in name else name
    name = "CatBoostRegressor" if "CatBoostRegressor" in name else name
    fit_param = {"X": X_train, "y": y_train}
    # if name in ["XGBRegressor", "CatBoostRegressor", "LGBMRegressor"]:
    #     fit_param["eval_set"] = [(X_valid, y_valid)]
    #     if name in ["CatBoostRegressor"]:
    #         fit_param["silent"] = True

    start = time.time()
    model.fit(**fit_param)
    train_time = time.time() - start
    y_pred = model.predict(X_valid)
    y_pred_exp_int = ((np.exp(y_pred) * 2 + 1) // 2).astype(int)    # 対数戻して、四捨五入int
    print(f"{name}, {train_time:.3f}s")
    scores.append([name, train_time, 
                   explained_variance_score(np.exp(y_valid), y_pred_exp_int),
                   mean_squared_error(np.exp(y_valid), y_pred_exp_int, squared=False), 
                   mean_absolute_error(np.exp(y_valid), y_pred_exp_int),
                   r2_score(np.exp(y_valid), y_pred_exp_int),
                   get_smape(np.exp(y_valid), y_pred_exp_int)])
df_score = pd.DataFrame(scores, columns=["model", "train_time", "explained_variance", "mae", "rmse", "r2", "smape"])

/tmp/ipykernel_5338/1470986145.py:16: RuntimeWarning: overflow encountered in exp
  y_pred_exp_int = ((np.exp(y_pred) * 2 + 1) // 2).astype(int)    # 対数戻して、四捨五入int
/tmp/ipykernel_5338/1470986145.py:16: RuntimeWarning: invalid value encountered in floor_divide
  y_pred_exp_int = ((np.exp(y_pred) * 2 + 1) // 2).astype(int)    # 対数戻して、四捨五入int
/tmp/ipykernel_5338/1470986145.py:16: RuntimeWarning: invalid value encountered in cast
  y_pred_exp_int = ((np.exp(y_pred) * 2 + 1) // 2).astype(int)    # 対数戻して、四捨五入int


LinearRegression, 0.540s
Lasso, 0.092s
DecisionTreeRegressor, 0.768s
Ridge, 0.052s
ElasticNet, 0.082s
RandomForestRegressor, 47.513s
XGBRegressor, 4.163s
Learning rate set to 0.082161
0:	learn: 1.0565430	total: 3.43ms	remaining: 3.43s
1:	learn: 0.9735991	total: 6.42ms	remaining: 3.2s
2:	learn: 0.8985799	total: 9.29ms	remaining: 3.09s
3:	learn: 0.8297291	total: 12.1ms	remaining: 3.02s
4:	learn: 0.7664972	total: 15.1ms	remaining: 2.99s
5:	learn: 0.7078641	total: 17.9ms	remaining: 2.97s
6:	learn: 0.6543742	total: 20.8ms	remaining: 2.95s
7:	learn: 0.6067160	total: 23.6ms	remaining: 2.93s
8:	learn: 0.5634808	total: 26.5ms	remaining: 2.92s
9:	learn: 0.5229007	total: 29.4ms	remaining: 2.91s
10:	learn: 0.4860836	total: 32.2ms	remaining: 2.9s
11:	learn: 0.4528982	total: 35.1ms	remaining: 2.89s
12:	learn: 0.4227642	total: 38.1ms	remaining: 2.89s
13:	learn: 0.3950602	total: 41ms	remaining: 2.88s
14:	learn: 0.3699453	total: 44ms	remaining: 2.89s
15:	learn: 0.3471650	total: 47.1ms	remaining: 2.9s
1

In [87]:
df_score.sort_values("smape", ascending=True)

,model,train_time,explained_variance,mae,rmse,r2,smape
7,CatBoostRegressor,3.116341,9.783197e-01,3.220695e+01,1.722875e+01,9.736836e-01,9.965704
8,LGBMRegressor,0.264144,9.755026e-01,3.407647e+01,1.786663e+01,9.705398e-01,10.070713
10,HistGradientBoostingRegressor,1.638100,9.753132e-01,3.423111e+01,1.790327e+01,9.702718e-01,10.106009
6,XGBRegressor,4.162707,9.767978e-01,3.318206e+01,1.760354e+01,9.720660e-01,10.146105
5,RandomForestRegressor,47.512773,9.771875e-01,3.288201e+01,1.759872e+01,9.725689e-01,10.249666
9,GradientBoostingRegressor,10.469843,9.660201e-01,4.006379e+01,2.037958e+01,9.592778e-01,10.719073
2,DecisionTreeRegressor,0.767865,9.717171e-01,3.586826e+01,1.921863e+01,9.673602e-01,11.218350
0,LinearRegression,0.540498,-7.072360e+30,5.288517e+17,3.032341e+16,-7.095688e+30,14.402780
3,Ridge,0.052292,9.492418e-01,4.882348e+01,2.627317e+01,9.395238e-01,15.143136
1,Lasso,0.092458,0.000000e+00,2.157411e+02,1.277431e+02,-1.808441e-01,76.749820


In [30]:
def disp_scores(Y_test, test_pred):
    print("Test Scores")
    print("\tExplained variance:", explained_variance_score(Y_test, test_pred))
    print("\tMean absolute error:", mean_absolute_error(Y_test, test_pred))
    print("\tMean squared error:", mean_squared_error(Y_test, test_pred))
    print("\tR2 score:", r2_score(Y_test, test_pred))
    print("\tSMAPE score:", get_smape(Y_test, test_pred))

In [31]:
def get_prediction(train, valid, test, model):
    """各予測値を返す"""
    ret_arr = []
    for x in [train, valid, test]:
        prediction = model.predict(x)
        # # 小さい値で差分が出るのを抑えるため、四捨五入してみる    対数にしてたら、四捨五入ダメ
        # ret_arr.append(((prediction * 2 + 1) // 2).astype(int))
        ret_arr.append(prediction)
    return ret_arr

def learning(model, X_train, Y_train, X_valid, Y_valid, X):
    """学習、予測値出力"""
    model.fit(X_train, Y_train)
    train_pred, valid_pred, pred = get_prediction(X_train, X_valid, X, model)
    disp_scores(Y_valid, valid_pred)
    return (train_pred, valid_pred, pred)

In [32]:
def get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred):
    """学習とテスト確認用 DataFrame 作成"""
    cols = ["id", "date", "country", "store", "product"]
    # オリジナル
    df_plt = df_train[cols].copy()
    df_plt["num_sold"] = df_train["num_sold"]
    df_plt["kind"] = "orig"
    # 学習結果
    df_train_pred = pd.merge(X_train, df_train, left_index=True, right_index=True)[cols]
    df_train_pred["num_sold"] = np.exp(train_pred)    # 対数で学習させてたら、指数で戻す
    df_train_pred["kind"] = "train"
    # 検証結果
    df_valid_pred = pd.merge(X_valid, df_train, left_index=True, right_index=True)[cols]
    df_valid_pred["num_sold"] = np.exp(valid_pred)
    df_valid_pred["kind"] = "test"
    # 実際の予測
    df_pred = df_test.copy()
    df_pred["num_sold"] = np.exp(pred)
    df_pred["kind"] = "pred"

    df_plt = pd.concat([df_plt, df_train_pred, df_valid_pred, df_pred]).reset_index(drop=True)
    # print(df_plt.shape)
    return df_plt

In [123]:
## RandomForest
model_name = "RandomForest"
model = RandomForestRegressor()
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9926548603850557
	Mean absolute error: 0.10279542952759994
	Mean squared error: 0.016610505576286485
	R2 score: 0.9879665416451708
	SMAPE score: 2.5093895830919326


In [33]:
def check_correlation(df_plt):
    """国、店舗、製品毎の相関確認"""
    df_buf = df_plt[df_plt["kind"]=="test"].drop(columns=["kind"])
    df_buf.rename(columns={"num_sold": "num_sold_test"}, inplace=True)
    df_buf = pd.merge(df_buf, df_plt[df_plt["kind"]=="orig"].drop(columns=["kind"]))

    buf = []
    for c in df_buf["country"].unique():
        for s in df_buf["store"].unique():
            for p in df_buf["product"].unique():
                df_p = df_buf[(df_buf["country"]==c) & (df_buf["store"]==s) & (df_buf["product"]==p)]
                rmse = mean_squared_error(df_p["num_sold"], df_p["num_sold_test"], squared=False)
                smape = get_smape(df_p["num_sold"], df_p["num_sold_test"])
                buf.append([c, s, p, rmse, smape])
    df_ret = pd.DataFrame(buf, columns=["country", "store", "product", "rmse", "smape"])
    return df_ret

In [37]:
check_correlation(df_plt).sort_values("smape")

,country,store,product,rmse,smape
68,Spain,Kaggle Store,Using LLMs to Win More Kaggle Competitions,7.107203,4.915174
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,30.800685,4.957550
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,26.235812,4.999434
50,Japan,Kaggle Store,Using LLMs to Improve Your Coding,11.268090,5.034321
63,Spain,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,4.825162,5.038845
...,...,...,...,...,...
31,Estonia,Kaggle Learn,Using LLMs to Train More LLMs,13.931928,18.548533
30,Estonia,Kaggle Learn,Using LLMs to Improve Your Coding,14.624337,18.769614
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.188668,18.815640
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,78.860133,18.893183


In [90]:
## LightGBM
model_name = "LightGBM"
model = LGBMRegressor()
# model = LGBMRegressor(data_sample_strategy='goss', max_depth=15, n_estimators=150, num_leaves=31, force_row_wise=True)    # warning 抑制
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 627
[LightGBM] [Info] Number of data points in the train set: 82125, number of used features: 107
[LightGBM] [Info] Start training from score 4.539447
Test Scores
	Explained variance: 0.9928228147152496
	Mean absolute error: 0.10155290538313574
	Mean squared error: 0.016158934870384226
	R2 score: 0.9882936814338292
	SMAPE score: 2.4497572565937524


In [35]:
df_corr = check_correlation(df_plt)
pd.concat([df_corr.sort_values("smape").head(3), df_corr.sort_values("smape").tail(3)])

,country,store,product,rmse,smape
45,Japan,Kaggle Learn,Using LLMs to Improve Your Coding,6.697342,4.794616
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,31.724727,4.829026
71,Spain,Kagglazon,Using LLMs to Train More LLMs,30.923053,4.831571
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.480270,19.592509
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,81.412668,19.676335
37,Estonia,Kaggle Store,Using LLMs to Win Friends and Influence People,3.517119,20.324771


In [36]:
csp, df_buf = select_csp(df_plt, [2, 1, 2])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [37]:
df_corr.groupby(by=["country"])["smape"].mean()

country
Argentina     8.572729
Canada       12.789617
Estonia      18.635249
Japan         5.247979
Spain         5.267670
Name: smape, dtype: float64

In [72]:
## Category Boosting
model_name = "CatBoost"
cat_params = {
    'n_estimators': 195,
    'learning_rate': 0.07725732658711602,
    'depth': 7,
    'l2_leaf_reg': 8.601133541582584,
    'subsample': 0.4279526734063217,
    'colsample_bylevel': 0.6767696482697301,
    'random_state': 42,
    'verbose': False
}
model = CatBoostRegressor(**cat_params)
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9930965440177393
	Mean absolute error: 0.10069115295812177
	Mean squared error: 0.015886431067984992
	R2 score: 0.9884910964458313
	SMAPE score: 2.4317676454764037


In [73]:
## Category Boosting
model_name = "CatBoost"
cat_params = {'verbose':False}
model = CatBoostRegressor(**cat_params)
train_pred, valid_pred, pred = learning(model, X_train, y_train, X_valid, y_valid, X_test)
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

Test Scores
	Explained variance: 0.9934836677837278
	Mean absolute error: 0.10014414811157739
	Mean squared error: 0.015440049398241128
	R2 score: 0.9888144770442455
	SMAPE score: 2.4365556634622965


In [39]:
df_corr = check_correlation(df_plt)
pd.concat([df_corr.sort_values("smape").head(3), df_corr.sort_values("smape").tail(3)])

,country,store,product,rmse,smape
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,28.698196,4.636911
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,24.500423,4.669645
55,Japan,Kagglazon,Using LLMs to Improve Your Coding,36.724602,4.684573
32,Estonia,Kaggle Learn,Using LLMs to Win Friends and Influence People,2.172067,18.992773
37,Estonia,Kaggle Store,Using LLMs to Win Friends and Influence People,3.337138,19.050035
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.214325,19.095518


In [41]:
# csp, df_buf = select_csp(df_plt, [2, 0, 4])
csp, df_buf = select_csp(df_plt, [2, 0, 2])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [42]:
df_corr.groupby(by=["country"])["smape"].mean()

country
Argentina     8.488073
Canada       12.902386
Estonia      18.524097
Japan         5.008760
Spain         5.027324
Name: smape, dtype: float64

---
### submit
- 2021年まで含んでモデル生成

In [91]:
print(f'{X_train["year"].unique()} \n{Xt_train["year"].unique()}')

[2017 2018 2019] 
[2017 2018 2019 2021]


In [92]:
print(model_name)
train_pred, valid_pred, pred = learning(model, Xt_train, yt_train, Xt_train, yt_train, X_test)
print(len(train_pred), len(valid_pred), len(pred))

LightGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005154 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 109500, number of used features: 108
[LightGBM] [Info] Start training from score 4.551155
Test Scores
	Explained variance: 0.9970436546876305
	Mean absolute error: 0.04946267970825107
	Mean squared error: 0.003941936822713056
	R2 score: 0.9970436546876303
	SMAPE score: 1.2260261726330208
109500 109500 27375


COVID-19 回復率反映

In [93]:
# 2022年の倍率予測のため、2020年以前と2020年と2021年の比率を確認
bef = df_all[df_all["year"]<2020].groupby(by=["country", "store", "product"])["num_sold"].median().rename("bef")
cov = df_all[df_all["year"]==2020].groupby(by=["country", "store", "product"])["num_sold"].median().rename("cov")
aft = df_all[df_all["year"]==2021].groupby(by=["country", "store", "product"])["num_sold"].median().rename("aft")
df_buf = pd.concat([bef, cov, aft], axis=1).reset_index()
df_buf["bef_ratio"] = df_buf["cov"] / df_buf["bef"]
df_buf["aft_ratio"] = df_buf["aft"] / df_buf["cov"]

In [77]:
df_buf.groupby("country")[["bef_ratio", "aft_ratio"]].mean()

,bef_ratio,aft_ratio
country,,
Argentina,0.660786,1.322779
Canada,0.886838,1.276801
Estonia,0.998285,1.249566
Japan,0.954490,1.045555
Spain,0.862986,1.184186


In [185]:
df_buf.groupby("store")[["bef_ratio", "aft_ratio"]].mean()

,bef_ratio,aft_ratio
store,,
Kagglazon,0.874077,1.211390
Kaggle Learn,0.868645,1.220903
Kaggle Store,0.875309,1.215039


In [186]:
df_buf.groupby("product")[["bef_ratio", "aft_ratio"]].mean()

,bef_ratio,aft_ratio
product,,
Using LLMs to Improve Your Coding,0.883903,1.221521
Using LLMs to Train More LLMs,0.901439,1.135175
Using LLMs to Win Friends and Influence People,0.882776,1.177330
Using LLMs to Win More Kaggle Competitions,0.854535,1.252726
Using LLMs to Write Better,0.840732,1.292136


↑ 比べると国毎の回復率が異なる。

In [94]:
df_country_ratio = df_buf.groupby("country")["aft_ratio"].mean().reset_index()
df_country_ratio

,country,aft_ratio
0,Argentina,1.322779
1,Canada,1.276801
2,Estonia,1.249566
3,Japan,1.045555
4,Spain,1.184186


In [95]:
# 対数を戻して、国毎の倍率反映
pred_exp = np.exp(pred)
for country, ratio in df_country_ratio.values:
    pred_exp[df_test["country"]==country] *= ratio

# 四捨五入
pred_exp_int = ((pred_exp * 2 + 1) // 2).astype(int)

In [96]:
# 更に掛ける (何故かスコアが良くなる。。。)
# covid_coef = 1.2    # score:35.40569 (catboost)
# covid_coef = 1.5    # やり過ぎ score:39.94571 (catboost)
covid_coef = 1.3    # score:34.66034 (lgbm)
pred_exp_int = pred_exp_int * covid_coef

In [97]:
df_plt = df_train.copy()
df_plt["kind"] = "origin"

df_buf = Xt_train.copy()
df_buf["num_sold"] = np.exp(train_pred)    # 2020年無いので、一旦Xt_trainへ移す
df_buf = df_train.drop(columns=["num_sold"]).join(df_buf[["num_sold"]], how="inner")
df_buf["kind"] = "train"
df_plt = pd.concat([df_plt, df_buf[df_plt.columns]])

df_buf = df_test.copy()
df_buf["num_sold"] = pred_exp_int
df_buf["kind"] = "pred"
df_plt = pd.concat([df_plt, df_buf[df_plt.columns]])

In [98]:
csp, df_buf = select_csp(df_plt, [0, 0, 0])
# csp, df_buf = select_csp(df_plt, [4, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [99]:
# df_subm = pd.merge(df_subm.drop(columns=["num_sold"]), df_test[["id", "num_sold"]].copy(), how="left")
df_subm["num_sold"] = pred_exp_int
print("num of Nan:", df_subm["num_sold"].isna().sum())
df_subm.describe()["num_sold"]

num of Nan: 0


count    27375.000000
mean       276.520614
std        306.119510
min          6.500000
25%         78.000000
50%        163.800000
75%        310.700000
max       2091.700000
Name: num_sold, dtype: float64

In [100]:
df_subm

,id,num_sold
0,136950,63.7
1,136951,61.1
2,136952,9.1
3,136953,57.2
4,136954,52.0
...,...,...
27370,164320,1170.0
27371,164321,1170.0
27372,164322,184.6
27373,164323,1089.4


In [101]:
df_subm.to_csv("./submission.csv", index=False)